## Imports

In [ ]:
!git clone https://github.com/daniel-furman/Capstone.git
!pip install -r /content/Capstone/requirements.txt

Cloning into 'Capstone'...
remote: Enumerating objects: 2869, done.
remote: Counting objects: 100% (543/543), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 2869 (delta 375), reused 517 (delta 349), pack-reused 2326
Receiving objects: 100% (2869/2869), 270.68 MiB | 16.94 MiB/s, done.
Resolving deltas: 100% (1827/1827), done.
Updating files: 100% (156/156), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 760.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 9.3 MB/s eta 

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python3 --version

Python 3.9.16


In [ ]:
import openai

import re
import datetime
import time
import pandas as pd
import os
import tqdm
from argparse import ArgumentParser
from argparse import Namespace

from datasets import load_dataset

## Pull in openai api key

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
key_path = (
    "/content/drive/MyDrive/Colab Files/open_ai_api_key/open_ai_personal_api_creds.txt"
)

In [ ]:
api_cred = open(key_path, "r").read()

## Load Data

In [ ]:
# grab data
dataset = load_dataset("CalibraGPT/Fact-Completion", split="English")

Extracting data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating Spanish split:   0%|          | 0/18786 [00:00<?, ? examples/s]

Generating Slovenian split:   0%|          | 0/7873 [00:00<?, ? examples/s]

Generating French split:   0%|          | 0/18395 [00:00<?, ? examples/s]

Generating Romanian split:   0%|          | 0/17568 [00:00<?, ? examples/s]

...

Generating Russian split:   0%|          | 0/3289 [00:00<?, ? examples/s]

Generating Croatian split:   0%|          | 0/7358 [00:00<?, ? examples/s]

Generating Polish split:   0%|          | 0/9484 [00:00<?, ? examples/s]

Generating Serbian split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/CalibraGPT___parquet/CalibraGPT--Fact-Completion-863b8999400e9abf/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [ ]:
# put facts into a list
entries = []
for entry in dataset:
    entries.append(entry)

## Explore API

For GPT-3 we use the OpenAI API, which provides access to four
model variants:

* `Ada`  Small (2.7 billion
parameters)
* `Babbage` Medium (6.7 billion)
* `Curie` Large (13 billion)
* `Davinci` X-Large (175 billion). 

In [ ]:
debug = False
model_variant = "text-davinci-003"
openai.api_key = api_cred
df_list = []


for entry in entries[:2]:
    if debug:
        print(f"stem: {entry['stem']}")
        print(f"true: {entry['true']}")
        print(f"false: {entry['false']}")

    stem = entry["stem"]
    true = entry["true"]
    false = entry["false"]  # todo generalize to false-list case

    true_false_to_check = [true, false]

    tokens_to_generate = len(max(true_false_to_check, key=len)) + 2
    print(f"stem: {stem}")
    while True:
        try:
            res = openai.Completion.create(
                model=model_variant,
                prompt=stem,
                max_tokens=1,
                logprobs=100,
                temperature=0,
                echo=True,
            )
            break
        except:
            print("pausing")
            time.sleep(1)
            continue

    df = pd.DataFrame(res["choices"][0]["logprobs"])

    print(f"top 5 highest predicted tokens: {list(df['top_logprobs'])[-1]}")

stem: What sport does Megan Rapinoe play? They play
top 5 highest predicted tokens: {
  "\n": -3.3882465,
  " Soccer": -2.1739237,
  " association": -3.7330327,
  " football": -3.462209,
  " soccer": -0.30829552
}
stem: Trailer Park Boys was from
top 5 highest predicted tokens: {
  " 1999": -1.2222556,
  " 2001": -0.78234667,
  " Canada": -2.2324924,
  " a": -3.1904247,
  " the": -3.0567017
}
